Импорт необходимых библиотек

In [ ]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

- `os`: предоставляет функции для взаимодействия с операционной системой (пути, файлы)
- `cv2`: OpenCV, используется для обработки изображений (чтение, изменение размера и т.д.)
- `matplotlib.pyplot` (plt): инструмент для построения графиков и отображения изображений
- `numpy` (np): эффективная работа с многомерными массивами и числовыми вычислениями
- `tensorflow` (tf): фреймворк для построения и обучения моделей машинного обучения
- `keras`: модуль TensorFlow для упрощенного создания и управления нейронными сетями
- `fashion_mnist`: встроенный набор данных с изображениями одежды (60,000 + 10,000 примеров)
- Импорт из `tensorflow.keras.models`:
  - `Sequential`: класс для последовательного добавления слоев в модель
  - `load_model`: функция для загрузки сохраненных моделей
- Импорт из `tensorflow.keras.layers`:
  - `Dense`: полносвязный слой
  - `BatchNormalization`: слой нормализации входных данных
  - `GlobalAveragePooling2D`: слой субдискретизации для уменьшения размерности
- Импорт из `tensorflow.keras.utils`:
  - `to_categorical`: метод для преобразования каждого значения в вектор
- Импорт из `tensorflow.keras.callbacks`:
  - `EarlyStopping`: необходимо для установки критериев остановки обучения в случае непредвиденной ситуации

Загрузка и подготовка данных

In [ ]:
# Загрузка данных Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Список классов (объединены некоторые классы для упрощения)
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Shirt/Coat', 'Bag', 'Ankle boot/Sandal/Sneaker']

# Нормализация изображений (приведение значений пикселей к диапазону [0, 1])
x_train = x_train / 255.0
x_test = x_test / 255.0

# Добавление 3 каналов (RGB) и изменение размера изображений до 32x32
x_train = np.stack([x_train] * 3, axis=-1)  # (60000, 28, 28, 3)
x_test = np.stack([x_test] * 3, axis=-1)  # (10000, 28, 28, 3)

x_train = tf.image.resize(x_train, (32, 32))  # Resize до 32x32
x_test = tf.image.resize(x_test, (32, 32))  # Resize до 32x32

- `fashion_mnist.load_data()`: загружает набор данных Fashion MNIST
  - Возвращает кортеж из двух пар:
    - `(x_train, y_train)`: 60,000 обучающих изображений и меток
    - `(x_test, y_test)`: 10,000 тестовых изображений и меток
- `np.stack([x_train] * 3, axis=-1)`: Преобразует одноканальные изображения (градации серого) в трехканальные (RGB), дублируя один канал три раза.
- `tf.image.resize(x_train, (32, 32))`: Изменяет размер изображений обучающей выборки до 32x32 пикселей. Это необходимо, так как предобученные модели, такие как ResNet, ожидают изображения определенного размера (обычно 32x32 или больше).

Объединение классов

    -раньше у нас было 10 классов в fashion_mnist теперь будет 7, для этого надо объединить некоторые классы в 1

In [ ]:
# Объединение классов (переопределение меток)
y_train_copy = y_train.copy()
y_test_copy = y_test.copy()

for i in range(len(y_train)):
    if y_train_copy[i] == 5:
        y_train_copy[i] = 6
    elif y_train_copy[i] == 6:
        y_train_copy[i] = 4
    elif y_train_copy[i] == 7:
        y_train_copy[i] = 6
    elif y_train_copy[i] == 8:
        y_train_copy[i] = 5
    elif y_train_copy[i] == 9:
        y_train_copy[i] = 6

for i in range(len(y_test)):
    if y_test_copy[i] == 5:
        y_test_copy[i] = 6
    elif y_test_copy[i] == 6:
        y_test_copy[i] = 4
    elif y_test_copy[i] == 7:
        y_test_copy[i] = 6
    elif y_test_copy[i] == 8:
        y_test_copy[i] = 5
    elif y_test_copy[i] == 9:
        y_test_copy[i] = 6

y_train = np.array(y_train_copy)
y_test = np.array(y_test_copy)

Преобразование меток в one-hot encoding

- `keras.utils.to_categorical`: Преобразует метки классов (целые числа) в one-hot encoding (бинарный формат). Это необходимо для задач классификации, где модель должна предсказать вероятность принадлежности к каждому классу.

In [ ]:
y_train = keras.utils.to_categorical(y_train, 7)
y_test = keras.utils.to_categorical(y_test, 7)

Визуализация данных

    - в этом блоке показываются примеры изображений

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i])
    plt.xlabel(class_names[np.argmax(y_train[i])])
plt.show()

Загрузка предобученной модели ResNet50 без верхнего слоя

- `ResNet50`: Загружает предобученную модель ResNet50, которая была обучена на наборе данных ImageNet. ResNet50 — это глубокая сверточная нейронная сеть, состоящая из 50 слоев.
- `base_model`: Загруженная модель ResNet50 без верхних слоев. Эта модель будет использоваться как основа для создания новой модели.

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False

Создание новой модели на основе ResNet50

- `Sequential`: линейный стек слоев, который позволяет последовательно добавлять слои для построения модели. Каждый слой имеет ровно один входной и один выходной тензор.

In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(96, activation='relu'),
    Dense(7, activation='softmax')
])

Компиляция модели

- `model.compile`: Компилирует модель, настраивая процесс обучения. Это включает в себя выбор оптимизатора, функции потерь и метрик для оценки производительности модели.
    - `optimizer=tf.keras.optimizers.Adam()`: Оптимизатор, который будет использоваться для обновления весов модели во время обучения.
- `loss='categorical_crossentropy'`: Функция потерь, которая будет использоваться для оценки ошибки модели во время обучения.
- `metrics=['accuracy']`: Метрика, которая будет использоваться для оценки производительности модели.
- `model.summary`: Выводит сводную информацию о структуре модели, включая количество параметров в каждом слое и общее количество обучаемых параметров.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Вывод структуры модели
model.summary()

Добавляем условия остановки обучения

- `EarlyStopping`: Callback (обратный вызов), который останавливает обучение модели, если выбранная метрика перестает улучшаться. Это помогает предотвратить переобучение и сэкономить время и ресурсы.

In [ ]:

early_stopping = EarlyStopping(
    monitor='val_loss',  # Метрика, которую будем отслеживать (потери на валидации)
    patience=30,          # Количество эпох без улучшения, после которых обучение остановится
    restore_best_weights=True  # Восстановление весов модели с лучшей эпохи
)

Обучение модели

- `model.fit`: Обучает модель на предоставленных данных.
- `model.save`: Сохраняет обученную модель в файл.
- `keras.models.load_model`: Загружает ранее сохраненную модель из файла.

In [ ]:
# Обучение модели (если модель еще не обучена)
if "fashion_trained_model_3.keras" not in os.listdir():
    history = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test), callbacks=[early_stopping])
    model.save("fashion_trained_model_3.keras")

    # Построение графика точности на обучающей и тестовой выборках
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Test Accuracy')
    plt.title('Training and Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
else:
    # Если модель уже обучена, загружаем её
    model = keras.models.load_model('fashion_trained_model_3.keras')

Оценка модели

- `model.evaluate`: Оценивает производительность модели на тестовых данных. Эта функция вычисляет значение функции потерь и метрик, указанных при компиляции модели, для предоставленных данных.

In [ ]:
# Оценка точности на тестовой выборке
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Визуализация результатов

- ` model.predict`: Выполняет предсказание классов для предоставленных данных. Эта функция возвращает вероятности принадлежности каждого примера к каждому классу.

In [ ]:
# Предсказание классов для тестовой выборки
predictions = model.predict(x_test)

# Визуализация примеров изображений с предсказанными и истинными классами
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[i])
    # Предсказание классов
    predicted_class = np.argmax(predictions[i])
    true_class = np.argmax(y_test[i])
    plt.xlabel(f'Predicted: {class_names[predicted_class]}, Actual: {class_names[true_class]}')
plt.show()